In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal
from llms import gemini
from llms import chatGPT
from sklearn.metrics.pairwise import cosine_similarity
import json
import time
# from lemmatize import lemmatize
x_chat = chatGPT()
x_gemini = gemini()
print('hi')



hi


In [2]:
df = pd.read_csv('../output/df_with_examples.csv')
df = df.dropna(subset=["Example1"])

In [3]:

job_cols = [[f"Example_Description{i}",f"Example_Job_Title{i}"] for i in range(1,4)]
for job in job_cols[0:]:
    for col in job:
        df = x_chat.run_batch_embeddings(df,col)

df.to_csv('../output/df_with_examples_embeddings.csv', index=False)

Processing batch 0 to 199
Processing batch 200 to 399
Processing batch 400 to 599
Processing batch 600 to 799
Processing batch 800 to 999
Processing batch 1000 to 1199
Processing batch 1200 to 1399
Processing batch 1400 to 1599
Processing batch 1600 to 1799
Processing batch 1800 to 1999
Processing batch 2000 to 2199
Processing batch 2200 to 2399
Processing batch 2400 to 2599
Processing batch 2600 to 2799
Processing final batch 2800 to 2922
Processing batch 0 to 199
Processing batch 200 to 399
Processing batch 400 to 599
Processing batch 600 to 799
Processing batch 800 to 999
Processing batch 1000 to 1199
Processing batch 1200 to 1399
Processing batch 1400 to 1599
Processing batch 1600 to 1799
Processing batch 1800 to 1999
Processing batch 2000 to 2199
Processing batch 2200 to 2399
Processing batch 2400 to 2599
Processing batch 2600 to 2799
Processing final batch 2800 to 2922
Processing batch 0 to 199
Processing batch 200 to 399
Processing batch 400 to 599
Processing batch 600 to 799
Pr

In [6]:
df_exp.description.iloc[2]

'Sales managers that determine price schedules and discount rates.'

In [ ]:
# random_descriptions = df_exp['description'].sample(10, random_state=2)
# print("\n".join(random_descriptions.values))


In [11]:
df_exp = pd.read_csv('../input/gpts_labels/gpt_exposure_embeddings.csv')

df_exp.task_embedding = df_exp.task_embedding.apply(lambda x: x.strip("[]").split(", "))

task_embeddings = []
for x in df_exp.task_embedding:
    task_embeddings.append([float(y) for y in x])
df_exp.task_embedding = task_embeddings


task_embeddings = np.array(task_embeddings)
task_embeddings = np.vstack(task_embeddings)


df_exp.title_embedding = df_exp.title_embedding.apply(lambda x: x.strip("[]").split(", "))
title_embeddings = []
for x in df_exp.title_embedding:
    title_embeddings.append([float(y) for y in x])
df_exp.title_embedding = title_embeddings


task_embeddings.shape

(11582, 3072)

In [7]:
df = pd.read_csv('../output/df_with_examples_embeddings.csv')


for i in range(3):
    df[f"Example_Description{i+1}_embedding"] = df[f"Example_Description{i+1}_embedding"].apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])
    df[f"Example_Job_Title{i+1}_embedding"] = df[f"Example_Job_Title{i+1}_embedding"].apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])




In [ ]:

# results = []
results = list(pd.read_csv("../output/onet_df.csv").values)

for index, row in list(df.iterrows())[900:]:
    print("***********************")
    print(f"Processing {index}")
    name = row["organization name"]
    print(name)
    print(row["generated_description"])
        
    job_results = []
    for i in range(3): 
        example = row[f"Example{i+1}"]
        description = row[f'Example_Description{i+1}']
        job_title = row[f'Example_Job_Title{i+1}']
        print(f"\nProcessing Example: {example} \nJob: {description}\n")

        job_embedding = np.array(row[f'Example_Description{i+1}_embedding']).reshape(1,-1)
        job_title_embedding = np.array(row[f'Example_Job_Title{i+1}_embedding']).reshape(1,-1)
        cosine_sim = cosine_similarity(job_embedding, task_embeddings, 'cosine')
        
        top3 = sorted(zip(cosine_sim[0], df_exp[['title', 'Task', 'description','title_embedding']].values), reverse=True)[:3]
        for cosine_sim_job, onet in top3:
            print(cosine_sim_job)
            onet_title_embedding = np.array([onet[3]])
            cosine_sim_title = cosine_similarity(job_title_embedding, onet_title_embedding, 'cosine')[0][0]
            print(f"{onet[2]} \nCosine Similarity: {cosine_sim_job}, Person Cosine Similarity: {cosine_sim_title}")
            results.append([name,example,row[f"situation{i+1}_conf_interval"], row[f"situation{i+1}_conf_interval_reasoning"],job_title,onet[0],onet[1],job_title, cosine_sim_job,cosine_sim_title])
 

    if index % 50 == 0:
        print("**Saving Results**\n\n")
        onet_df = pd.DataFrame(results, columns=["organization_name","example","situation_conf_interval", "situation_conf_interval_reasoning","job","onet_title","onet_task","example_job_title","task_similarity", "job_title_similarity"])
        onet_df.situation_conf_interval = onet_df.situation_conf_interval.astype("float")
        onet_df.to_csv("../output/onet_df.csv", index=False)
        time.sleep(10)
        
    print("***********************\n\n")


onet_df = pd.DataFrame(results, columns=["organization_name","example","situation_conf_interval", "situation_conf_interval_reasoning","job","onet_title","onet_task","example_job_title","task_similarity", "job_title_similarity"])
onet_df.situation_conf_interval = onet_df.situation_conf_interval.astype("float")
onet_df.to_csv("../output/onet_df.csv", index=False)

***********************
Processing 900
Cortip
Cortip is a consulting service that helps companies transition from agency-based software development to in-house contractor teams. They provide expertise in building, managing, and streamlining remote developer teams, focusing on finding the best talent, optimizing workflows, and implementing best practices.

Processing Example: A software startup is growing rapidly and wants to transition from using an agency for development to building their own in-house team. They lack the expertise and resources to effectively manage remote developers and build a sustainable development process. 
Job: Software Development Managers that recruit, hire, and train technical personnel.

0.6759318025848722
Software developers that supervise and assign work to programmers, designers, technologists, technicians, or other engineering or scientific personnel. 
Cosine Similarity: 0.6759318025848722, Person Cosine Similarity: 0.7096770782099392
0.6703704564941435


In [55]:
onet_df.iloc[0]

organization_name                                                          Luming Labs
example                              A research scientist at a pharmaceutical compa...
situation_conf_interval                                                            9.0
situation_conf_interval_reasoning    The scientist needs to analyze large datasets ...
job                                         [Example_Description3, Example_Job_Title3]
onet_title                                                   Natural Sciences Managers
onet_task                            Plan or direct research, development, or produ...
example_job_title                                                  Research Scientists
task_similarity                                                               0.656189
job_title_similarity                                                          0.583292
Name: 0, dtype: object

In [9]:
curr = pd.read_csv("../output/onet_df.csv")

print(curr.iloc[-1])
df[df["organization name"]=="Obviously AI"]



organization_name                                                         Obviously AI
example                              A software developer wants to create a chatbot...
situation_conf_interval                                                            7.0
situation_conf_interval_reasoning    This example is less confident as the task is ...
job                                     ['Example_Description3', 'Example_Job_Title3']
onet_title                                                         Software Developers
onet_task                            Confer with systems analysts, engineers, progr...
example_job_title                                                  Software Developers
task_similarity                                                               0.707505
job_title_similarity                                                          0.999999
Name: 8099, dtype: object


,organization name,num employees,founded date,full_description,industries,headquarters location,description,cb rank,postal code,headquarters region,...,Example_Job_Title3,Example_Description3,situation3_conf_interval,situation3_conf_interval_reasoning,Example_Description1_embedding,Example_Job_Title1_embedding,Example_Description2_embedding,Example_Job_Title2_embedding,Example_Description3_embedding,Example_Job_Title3_embedding
899,Obviously AI,1-10,2020-02-02,"At Obviously AI, our vision is to turn every c...","Artificial Intelligence (AI), Machine Learning...","San Francisco, California, United States",Obviously AI enables businesses to build custo...,"3,959",94105,www.obviously.ai,...,Software Developers,"Software Developers that design, develop, or t...",7.0,This example is less confident as the task is ...,"[-0.016252316534519196, -0.027554688975214958,...","[-0.005456793587654829, -0.015291607938706875,...","[-0.03545496240258217, -0.025911446660757065, ...","[-0.018091563135385513, -0.01092542428523302, ...","[-0.008595336228609085, 0.039918918162584305, ...","[-0.002831327961757779, 0.02374046854674816, -..."


In [10]:
s1 = "Animators and Multimedia Artists that create storyboards to illustrate scenes or action sequences for films, television programs, or other media."
s2 = "Special effects artists and animators that apply story development, directing, cinematography, and editing to animation to create storyboards that show the flow of the animation and map out key scenes and characters."



In [13]:
e1 = x_chat.get_embedding(s1)
#e1 = df_exp[df_exp.description.str.contains("managers that evaluate the organization")].task_embedding.iloc[0]
e2 = x_chat.get_embedding(s2)

#e2 = x_chat.get_embedding("Computer Information Systems Managers evaluate cloud service offering different provider select suitable option company 's need")
e1 = np.array(e1).reshape(1,-1)
e2 = np.array(e2).reshape(1,-1)
cosine_similarity(e1,e2, 'cosine')[0][0]

0.6625274673336069

In [17]:
title_embeddings = np.array(df_exp.title_embedding)
title_embeddings = np.vstack(title_embeddings)

In [147]:
onet_df = pd.read_csv("../output/results v3/onet_df.csv")
onet_df = onet_df[[col for col in onet_df.columns if "titles" not in col]]
onet_df[["Job_similarity_titles"]]= 'N/A'
len(onet_df)

6093